In [ ]:
import torch
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.insert(0, module_path)
print(sys.path)

import Learning.autoencoders as autoencoders
import Double_Pendulum.robot_parameters as robot_parameters
import Double_Pendulum.dynamics as dynamics


rp = robot_parameters.LUMPED_PARAMETERS
rp["m1"] = 0.
model = autoencoders.Analytic_transformer(rp)

In [ ]:
def analytic_theta_1(rp: dict, q):
    
    # h1 is defined as the length between the actuator attachment point and the mass of the double pendulum
    
    Rx = rp["xa"] - rp["l1"] * torch.cos(q[0]) - rp["l2"] * torch.cos(q[1])
    Ry = rp["ya"] - rp["l1"] * torch.sin(q[0]) - rp["l2"] * torch.sin(q[1])
    
    th1 = torch.sqrt(Rx**2 + Ry**2)
    
    return th1
    
    
def analytic_theta_2(rp: dict, q):
    
    # h2 is defined as the arctan between the vector from mass of double pendulum 
    # to the actuator point. 
    # This was Cosimo's hunch, and has been verified in the Mathematica code
    
    Rx = rp["xa"] - rp["l1"] * torch.cos(q[0]) - rp["l2"] * torch.cos(q[1])
    Ry = rp["ya"] - rp["l1"] * torch.sin(q[0]) - rp["l2"] * torch.sin(q[1])    
    
    th2 = torch.atan2(Ry,Rx)
    
    return th2

def analytic_theta(rp:dict, q):
    th1 = analytic_theta_1(rp, q)
    th2 = analytic_theta_2(rp, q)

    th = torch.stack([th1, th2], dim=-1)

    return th

In [ ]:
def analytic_inverse(rp: dict, th):

    """
    Inverse kinematics from theta to q, based on the end-effector
    position (xend, yend). 
    Returns a tuple with two sets of joint angles, one for clockwise
    and one for counter-clockwise configuration.
    """

    # Obtain end effector position.

    xend = rp["xa"] - th[0]*torch.cos(th[1])
    yend = rp["ya"] - th[0]*torch.sin(th[1])

    # Calculate the inside angle of the two joints, used to determine q1. Epsilon prevents NaN.
    epsilon = 0.00001

    numerator = (xend**2 + yend**2 - rp["l1"]**2 - rp["l2"]**2)
    denominator = torch.tensor(2*rp["l1"]*rp["l2"])
    fraction = numerator/denominator

    """
    if torch.abs(fraction - 1) > 1.1:
        raise ValueError("End effector outside of robot reach, inverse cannot be calculated")
    else:
    """
    
    epsilon = 1e-6
    fraction = torch.clamp(fraction, -1.0 + epsilon, 1.0 - epsilon)

    beta = torch.arccos(fraction)

    # Determine primary angles.
    q1 = torch.atan2(yend, xend + epsilon) - torch.atan2(rp["l2"]*torch.sin(beta), epsilon + rp["l1"] + rp["l2"]*torch.cos(beta))
    q2 = q1 + beta

    # Determine secondary angles.
    q1_alt = torch.atan2(yend, xend) + torch.atan2(rp["l2"]*torch.sin(beta), epsilon + rp["l1"] + rp["l2"]*torch.cos(beta))
    q2_alt = q1_alt - beta 

    # Normalize values between -pi and pi.
    q1 = (q1 + torch.pi) % (2 * torch.pi) - torch.pi
    q2 = (q2 + torch.pi) % (2 * torch.pi) - torch.pi
    q1_alt = (q1_alt + torch.pi) % (2 * torch.pi) - torch.pi
    q2_alt = (q2_alt + torch.pi) % (2 * torch.pi) - torch.pi

    q = torch.stack([q1, q2], dim=-1)
    q_alt = torch.stack([q1_alt, q2_alt], dim=-1)

    # Check whether the primary angle is clockwise. Otherwise, swap with secondary.
    q_cw = q
    q_ccw = q_alt
    
    return q_cw, q_ccw

In [ ]:
q = torch.tensor([[0.2, 0.3]]).requires_grad_(True)
q_d = torch.tensor([[0.1, 0.4]]).requires_grad_(True)

th = model.encoder(q.squeeze(0)).unsqueeze(0)
#th = analytic_theta(rp, q)
th_copy = th.detach().clone().requires_grad_(True)
q_hat = model.decoder(th_copy, clockwise = False)

J_forward = model.jacobian_enc(q, clockwise = False)
J_inverse = model.jacobian_dec(th, clockwise = False)
J_inverse_copy = model.jacobian_dec(th_copy, clockwise = False)
J_inverse_copy_trans = torch.transpose(J_inverse_copy, 0, 1)

th_d = (J_forward @ q_d.T).T
th_d_copy = th_d.detach().clone().requires_grad_(True)
q_d_hat = (J_inverse_copy @ th_d_copy.T).T

print(torch.autograd.grad(q_hat[0, 0], th_copy, create_graph=True))


print("th_copy gradient?", th_copy.requires_grad)
print("J_inv_copy gradient?", J_inverse_copy.requires_grad)

print("th_copy:", th_copy)
print("q size:", q.size())
print("q_hat size:", q_hat.size())

print("q_hat_squeeze size:", q_hat.squeeze(0).size())
Mq, Cq, Gq = dynamics.dynamical_matrices(rp, q_hat.squeeze(0), q_d_hat.squeeze(0))
#print(torch.autograd.grad(Mq[0, 1], th_copy))
print("Mq:\n", Mq)

#print(torch.autograd.grad(J_inverse_copy_trans[1,1], th_copy))

Mth = J_inverse_copy_trans @ Mq @ J_inverse_copy
print("Mth\n", Mth)
M1 = Mth[1,1]
print(M1)
dM00dth = torch.autograd.grad(Mth[0,0], th_copy, create_graph=True)[0]
dM01dth = torch.autograd.grad(Mth[0,1], th_copy, create_graph=True)[0]
dM10dth = torch.autograd.grad(Mth[1,0], th_copy, create_graph=True)[0]
dM11dth = torch.autograd.grad(Mth[1,1], th_copy, create_graph=True)[0]
print(dM00dth)
print(dM01dth)
print(dM10dth)
print(dM11dth)


In [ ]:
print(rp["m2"] * 2*th[0,0])